# Reports

In [8]:
import os
import re
from datetime import datetime
current_datetime = datetime.now().strftime('%Y%m%d.%H%M')

import numpy as np
import pandas as pd

# for the notebook rendering 
from IPython.display import display, HTML, FileLink, Markdown
from IPython.display import FileLink as FL
from IPython.display import Markdown as MD


DIR = os.getcwd()
print(f'{DIR=}')

RDIR = os.path.join(DIR,'reports')
print(f'{RDIR=}')

DIR='c:\\Users\\JGarza\\GitHub\\wifi_monitor'
RDIR='c:\\Users\\JGarza\\GitHub\\wifi_monitor\\reports'


In [9]:
def break_down_buckets(idf,column,buckets,message='',nan_value=-1):
    """
    breaks a column down into buckets/bins
    """
    idf = idf.fillna(nan_value)

    print('',message,'\ncolumn: ',column, '\nbuckets: ', buckets)
    
    idf = pd.DataFrame(idf[column])
    idf['bucket'] = pd.cut(idf[column], bins=buckets)
    idf = idf.groupby(by='bucket').count()
    idf['percent'] = (idf[column]/idf[column].sum())*100
    idf['percent'] = idf['percent'].round(2)
    display(idf)

    return idf

#test
# break_down_buckets(df,'AGE_YRS',[-1,0,15,25,35,45,55,65,75,85,500])

In [10]:
def bar(num,denom=100.0,length=30,fillchar='#',emptychar='_'):
    fillnum = ((int)( (num/denom) * length))
    return '[' + ( fillnum * fillchar ).ljust(length,emptychar)  + ']' # + f" {(num/denom)*100.0:.2f}%     " 

In [11]:
# YYYYMMDDHHmm
DateFilter = None
DateFilter = 202501100000 # new firmware r44715
# DateFilter = 202501160000 # changed some settings in Error 404 NH
# DateFilter = 202501200000

In [12]:
def getfirmware(ssid,dt):
    if 'NH' in ssid:
        
        if dt <= 202501100810:
            return "r58881"
        elif dt > 202501100811 and dt <= 202501102101:
            return "r44715"
        elif dt > 202501102101 : #and dt <= 203000000000:
            return "r58881"
    else:
        return ""

In [13]:
data = {}

for i in os.listdir(RDIR):
    # print(i)
    try:

        text = ''
        with open(os.path.join(RDIR, i), 'r') as f:
            text = f.read()
        # print(text)

        dt = re.findall('Datetime_alt: .*\n',text)[0]
        dt = dt.replace('Datetime_alt: ','').replace('\\n','')
        dt = int(dt)
            
        if DateFilter != None:
            if dt < DateFilter:
                #move on to next
                print('exclude',dt,DateFilter)
                continue
            else:
                print('include',dt,DateFilter)

        wifi = re.findall('^SSID:.*\n',text)[0]
        wifi = wifi.replace('\n', '').replace('SSID: ','')
        # wifi = wifi + " " + getfirmware(wifi,dt)

        pings = re.findall('.*time=.*\n',text)
        pings = [ re.sub('.*time=','',p) for p in pings]
        pings = [ re.sub('ms.*\n','',p) for p in pings]
        pings = [ int(p) for p in pings]
        # print(*pings,sep='\n')


        signalstrength =  None
        try:
            signalstrength = re.findall('Signal_Strength:.*\n',text)[0]
            signalstrength = signalstrength.replace('\n', '').replace('Signal_Strength: ','').replace('%','')
        except:
            pass

        timeouts = re.findall('Request timed out.\n',text)

        trycount = re.findall('(Request|Reply).*\n',text)

        if wifi in data.keys():
            pass
        else:
            data[wifi] = {}
            data[wifi]['pings'] = []
            data[wifi]['timeouts'] = []
            data[wifi]['trycount'] = []
            data[wifi]['signalstrength'] = []
        
        data[wifi]['pings'].extend(pings)
        data[wifi]['timeouts'].extend([len(timeouts)])
        data[wifi]['trycount'].extend([len(trycount)])

        if signalstrength != None:
            data[wifi]['signalstrength'].append(int(signalstrength))

    except Exception as e:
        pass
        print(i,e)


Error404NH_202412292039.txt list index out of range
exclude 202412300625 202501100000
exclude 202412310635 202501100000
exclude 202412311659 202501100000
exclude 202501011257 202501100000
exclude 202501020539 202501100000
exclude 202501030416 202501100000
exclude 202501041159 202501100000
exclude 202501051039 202501100000
exclude 202501060604 202501100000
exclude 202501070520 202501100000
exclude 202501080600 202501100000
exclude 202501090551 202501100000
include 202501100810 202501100000
include 202501110123 202501100000
include 202501121229 202501100000
include 202501130644 202501100000
include 202501140522 202501100000
include 202501150633 202501100000
include 202501152246 202501100000
include 202501152256 202501100000
include 202501160552 202501100000
include 202501170804 202501100000
include 202501171155 202501100000
include 202501171205 202501100000
include 202501171329 202501100000
include 202501180755 202501100000
include 202501191342 202501100000
include 202501201050 202501100

In [14]:
file = os.path.join(DIR,'docs','index.html')
with open(file,'w',encoding='utf8') as f:

    f.write(f'<h>{current_datetime}</h>')

    for k in data.keys():
        
        f.write('<p>' + '-'*20 + '</p>')

        f.write(f'<p>{k}</p>')
        df_pings = pd.DataFrame(data[k]['pings'])

        csvlines = df_pings.describe().to_csv()
        f.write(csvlines[4::].replace('\n','<br \>').replace(',','\t'))

        f.write('<br \>')
        f.write(f'Signal Strength (min) {min(data[k]["signalstrength"])}<br \>')
        f.write(f'Signal Strength (mean) {sum(data[k]["signalstrength"])/len(data[k]["signalstrength"]):.2f}<br \>')
        f.write(f'Signal Strength (max) {max(data[k]["signalstrength"])}<br \>')

        # print(df_pings)

        bdb = break_down_buckets(df_pings,0,[0,5,10,15,20,30,40,50,500])
        bdb['bar'] = bdb.percent.apply(bar)
        # display(bdb)

        f.write('<br \>')
        f.write(bdb.to_html())
        f.write('<br \>')

        f.write(f'Request timed out (failed pings):<br \>')
        f.write(f'{ sum(data[k]['timeouts']) } out of { sum(data[k]['trycount']) }<br \>')
        f.write(f'{ sum(data[k]['timeouts']) / sum(data[k]['trycount']) }')

  
column:  0 
buckets:  [0, 5, 10, 15, 20, 30, 40, 50, 500]


<>:14: SyntaxWarning: invalid escape sequence '\>'
<>:16: SyntaxWarning: invalid escape sequence '\>'
<>:17: SyntaxWarning: invalid escape sequence '\>'
<>:18: SyntaxWarning: invalid escape sequence '\>'
<>:19: SyntaxWarning: invalid escape sequence '\>'
<>:27: SyntaxWarning: invalid escape sequence '\>'
<>:29: SyntaxWarning: invalid escape sequence '\>'
<>:31: SyntaxWarning: invalid escape sequence '\>'
<>:32: SyntaxWarning: invalid escape sequence '\>'
<>:14: SyntaxWarning: invalid escape sequence '\>'
<>:16: SyntaxWarning: invalid escape sequence '\>'
<>:17: SyntaxWarning: invalid escape sequence '\>'
<>:18: SyntaxWarning: invalid escape sequence '\>'
<>:19: SyntaxWarning: invalid escape sequence '\>'
<>:27: SyntaxWarning: invalid escape sequence '\>'
<>:29: SyntaxWarning: invalid escape sequence '\>'
<>:31: SyntaxWarning: invalid escape sequence '\>'
<>:32: SyntaxWarning: invalid escape sequence '\>'
C:\Users\JGarza\AppData\Local\Temp\ipykernel_1524\1694920660.py:14: SyntaxWarning:

,0,percent
bucket,,
"(0, 5]",0,0.00
"(5, 10]",387,17.14
"(10, 15]",1375,60.89
"(15, 20]",211,9.34
"(20, 30]",106,4.69
"(30, 40]",46,2.04
"(40, 50]",23,1.02
"(50, 500]",110,4.87


  
column:  0 
buckets:  [0, 5, 10, 15, 20, 30, 40, 50, 500]


C:\Users\JGarza\AppData\Local\Temp\ipykernel_1524\3656564501.py:11: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  idf = idf.groupby(by='bucket').count()


,0,percent
bucket,,
"(0, 5]",0,0.00
"(5, 10]",1082,47.23
"(10, 15]",1014,44.26
"(15, 20]",96,4.19
"(20, 30]",72,3.14
"(30, 40]",7,0.31
"(40, 50]",5,0.22
"(50, 500]",15,0.65


  
column:  0 
buckets:  [0, 5, 10, 15, 20, 30, 40, 50, 500]


C:\Users\JGarza\AppData\Local\Temp\ipykernel_1524\3656564501.py:11: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  idf = idf.groupby(by='bucket').count()


,0,percent
bucket,,
"(0, 5]",0,0.00
"(5, 10]",94,4.10
"(10, 15]",1529,66.74
"(15, 20]",409,17.85
"(20, 30]",174,7.59
"(30, 40]",37,1.62
"(40, 50]",16,0.70
"(50, 500]",32,1.40
